In [5]:
import urllib.request
import re
import pickle
from bs4 import BeautifulSoup

In [6]:
def gets_html(url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}):
    """ Lee un url y devuelve el código html """
    req = urllib.request.Request(url,headers=headers)
    return(urllib.request.urlopen(req).read().decode("utf-8"))

def find_links(soup,url_type):
    
    if url_type == "non_recipe":
        links = soup.find_all('a',class_="fixed-recipe-card__title-link",href=True)
    elif url_type == "recipe":
        links = soup.find_all('a', {"data-click-id":[re.compile("simslot_\d")]},href=True)
    return links

def filter_recipes_links(links):
    
    RECIPE_REGEX = re.compile('href=\"https://www.allrecipes.com/recipe/\d+/.*?\"')
    
    # Convert to string 
    string_links = [str(i) for i in links]

    # Filter by regex
    filtered_links = list(filter(RECIPE_REGEX.search, string_links))
    
    return filtered_links

def gets_links(links, n_links):
    """ Devuelve los primeros n links de un artículo """
    out = []
    while links and len(out) < n_links:
        li = links.pop(0)
        if 'http' in li:
            link_tmp = re.compile('href="(.+?)"').findall(li)[0]
        out.append(link_tmp)
    return(out)

def scraper(to_scrap, n_links, max_depth):

    html_data = {}
    link_regex = 'https://www.allrecipes.com/recipe/\d+'
    link_regex = re.compile(link_regex)
    
    while to_scrap:
        url, depth = to_scrap.pop()
        print(url,depth)
        if (url not in html_data) and (depth <= max_depth):
            if link_regex.match(url):
                URL_TYPE = "recipe"
            else:
                URL_TYPE = "non_recipe"
                
            html_data[url] = gets_html(url)
            soup = BeautifulSoup(html_data[url])
            
            links = find_links(soup,url_type=URL_TYPE)
            links = filter_recipes_links(links)
            
            page_urls = gets_links(links, n_links)
            page_urls = [(e, depth + 1) for e in page_urls]
            to_scrap.extend(page_urls)
            
    return(html_data)

In [7]:
from requests_html import HTMLSession
session = HTMLSession()

r = session.get('https://www.allrecipes.com/recipes/80/main-dish/')
[(recipe,0) for recipe in r.html.links if "main-dish" in recipe] + [('https://www.allrecipes.com/recipes/80/main-dish/',0)]

[('https://www.allrecipes.com/recipes/17212/everyday-cooking/seasonal/fall/main-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/15944/main-dish/curries/', 0),
 ('https://www.allrecipes.com/recipes/256/main-dish/meatloaf/', 0),
 ('https://www.allrecipes.com/recipes/17135/world-cuisine/asian/chinese/main-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/17504/world-cuisine/latin-american/mexican/main-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/248/main-dish/burgers/', 0),
 ('https://www.allrecipes.com/recipes/17515/everyday-cooking/quick-and-easy/main-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/1320/healthy-recipes/main-dishes/', 0),
 ('https://www.allrecipes.com/recipes/17137/world-cuisine/asian/thai/main-dishes/',
  0),
 ('https://www.allrecipes.com/recipes/15455/main-dish/meatballs/', 0),
 ('https://www.allrecipes.com/recipes/251/main-dish/sandwiches/', 0),
 ('https://www.allrecipes.com/recipes/674/main-dish/pork/pork-chops/', 0),
 ('https://www.allrecipe

In [9]:
to_scrap = [(recipe,0) for recipe in r.html.links if "main-dish" in recipe] + [('https://www.allrecipes.com/recipes/80/main-dish/',0)]
# Datos html
html_data = scraper(to_scrap, n_links=75, max_depth=20)

https://www.allrecipes.com/recipes/80/main-dish/ 0
https://www.allrecipes.com/recipe/64513/rosemary-ranch-chicken-kabobs/ 1
https://www.allrecipes.com/recipe/16700/salsa-chicken/ 1
https://www.allrecipes.com/recipe/141678/slow-cooker-pulled-pork/ 1
https://www.allrecipes.com/recipe/12720/grilled-salmon-i/ 1
https://www.allrecipes.com/recipe/51283/maple-salmon/ 1
https://www.allrecipes.com/recipe/8887/chicken-marsala/ 1
https://www.allrecipes.com/recipe/8652/garlic-chicken/ 1
https://www.allrecipes.com/recipe/16348/baked-pork-chops-i/ 1
https://www.allrecipes.com/recipe/65896/zesty-slow-cooker-chicken-barbecue/ 1
https://www.allrecipes.com/recipe/8669/chicken-cordon-bleu-ii/ 1
https://www.allrecipes.com/recipe/24264/sloppy-joes-ii/ 1
https://www.allrecipes.com/recipe/9023/baked-teriyaki-chicken/ 1
https://www.allrecipes.com/recipe/16354/easy-meatloaf/ 1
https://www.allrecipes.com/recipe/11758/baked-ziti-i/ 1
https://www.allrecipes.com/recipe/25203/brown-sugar-meatloaf/ 1
https://www.all

https://www.allrecipes.com/recipe/57319/pesto-cheesy-chicken-rolls/ 1
https://www.allrecipes.com/recipe/25300/manicotti-alla-romana/ 1
https://www.allrecipes.com/recipe/11836/stuffed-shells-i/ 1
https://www.allrecipes.com/recipe/72392/spinach-stuffed-chicken-breast/ 1
https://www.allrecipes.com/recipe/16189/cheeseburger-meatloaf/ 1
https://www.allrecipes.com/recipe/21532/stuffed-shells-iii/ 1
https://www.allrecipes.com/recipe/68380/garlic-lemon-double-stuffed-chicken/ 1
https://www.allrecipes.com/recipe/21190/stuffed-peppers-my-way/ 1
https://www.allrecipes.com/recipe/26661/stuffed-cabbage-rolls/ 1
https://www.allrecipes.com/recipe/8551/spinach-stuffed-chicken-breasts/ 1
https://www.allrecipes.com/recipe/16330/stuffed-peppers/ 1
https://www.allrecipes.com/recipe/8495/chicken-cordon-bleu-i/ 1
https://www.allrecipes.com/recipe/17991/stuffed-green-peppers-i/ 1
https://www.allrecipes.com/recipe/8669/chicken-cordon-bleu-ii/ 1
https://www.allrecipes.com/recipe/77194/bolognese-stuffed-bell-pe

https://www.allrecipes.com/recipe/228498/slow-cooker-baby-back-ribs/ 1
https://www.allrecipes.com/recipe/35753/scott-hibbs-amazing-whisky-grilled-baby-back-ribs/ 1
https://www.allrecipes.com/recipe/16448/simple-bbq-ribs/ 1
https://www.allrecipes.com/recipe/16091/baby-back-ribs/ 1
https://www.allrecipes.com/recipe/22230/slow-cooker-barbecue-ribs/ 1
https://www.allrecipes.com/recipe/266646/grilled-kiwi-and-chili-rubbed-short-ribs/ 1
https://www.allrecipes.com/recipe/267242/instant-pot-ribs/ 1
https://www.allrecipes.com/recipe/258488/botacks-fall-off-the-bone-pineapple-baked-rack-of-ribs/ 1
https://www.allrecipes.com/recipe/261791/peposa-dellimpruneta-tuscan-black-pepper-beef/ 1
https://www.allrecipes.com/recipe/262017/spicy-korean-ribs/ 1
https://www.allrecipes.com/recipe/231005/italian-sausage-baby-back-ribs/ 1
https://www.allrecipes.com/recipe/270527/instant-pot-galbi-korean-style-short-ribs/ 1
https://www.allrecipes.com/recipe/274040/chef-johns-copycat-mcrib-sandwich/ 1
https://www.al

https://www.allrecipes.com/recipe/73124/slow-cooker-salisbury-steak/ 1
https://www.allrecipes.com/recipe/17991/stuffed-green-peppers-i/ 1
https://www.allrecipes.com/recipe/20675/rempel-family-meatloaf/ 1
https://www.allrecipes.com/recipe/23567/slow-cooker-pepper-steak/ 1
https://www.allrecipes.com/recipe/21353/italian-spaghetti-sauce-with-meatballs/ 1
https://www.allrecipes.com/recipe/78370/hamburger-steak-with-onions-and-gravy/ 1
https://www.allrecipes.com/recipe/213742/meatball-nirvana/ 1
https://www.allrecipes.com/recipe/14595/salisbury-steak/ 1
https://www.allrecipes.com/recipe/16354/easy-meatloaf/ 1
https://www.allrecipes.com/recipe/25203/brown-sugar-meatloaf/ 1
https://www.allrecipes.com/recipe/16066/awesome-slow-cooker-pot-roast/ 1
https://www.allrecipes.com/recipe/57966/crispy-orange-beef/ 1
https://www.allrecipes.com/recipe/30575/blue-cheese-beef-tenderloin/ 1
https://www.allrecipes.com/recipe/132198/beef-tenderloin-with-ginger-shiitake-brown-butter/ 1
https://www.allrecipes.c

https://www.allrecipes.com/recipe/57966/crispy-orange-beef/ 1
https://www.allrecipes.com/recipe/201849/mongolian-beef-and-spring-onions/ 1
https://www.allrecipes.com/recipe/31144/sweet-and-sour-pork-iii/ 1
https://www.allrecipes.com/recipe/191156/restaurant-style-beef-and-broccoli/ 1
https://www.allrecipes.com/recipe/223382/chicken-stir-fry/ 1
https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/ 1
https://www.allrecipes.com/recipe/9027/kung-pao-chicken/ 1
https://www.allrecipes.com/recipe/86230/szechwan-shrimp/ 1
https://www.allrecipes.com/recipe/246165/spicy-cauliflower-fried-rice-with-pork/ 1
https://www.allrecipes.com/recipes/16954/main-dish/chicken/ 0
https://www.allrecipes.com/recipe/18775/aussie-chicken/ 1
https://www.allrecipes.com/recipe/8847/baked-honey-mustard-chicken/ 1
https://www.allrecipes.com/recipe/12009/creamy-cajun-chicken-pasta/ 1
https://www.allrecipes.com/recipe/8701/chicken-breasts-with-balsamic-vinegar-and-garlic/ 1
https://www.allrecipes.com/recipe

...

https://www.allrecipes.com/recipe/163808/pork-chops-with-black-cherry-sauce/ 1
https://www.allrecipes.com/recipe/31144/sweet-and-sour-pork-iii/ 1
https://www.allrecipes.com/recipe/75811/moms-stovetop-pork-ribs/ 1
https://www.allrecipes.com/recipe/143355/grilled-pork-chops-with-fresh-nectarine-salsa/ 1
https://www.allrecipes.com/recipe/237471/chef-johns-city-chicken/ 1
https://www.allrecipes.com/recipe/25467/ham-and-pineapple-kabobs/ 1
https://www.allrecipes.com/recipe/21174/bbq-pork-for-sandwiches/ 1
https://www.allrecipes.com/recipes/17245/main-dish/pasta/ 0
https://www.allrecipes.com/recipe/11815/penne-with-spicy-vodka-tomato-cream-sauce/ 1
https://www.allrecipes.com/recipe/20082/artichoke-spinach-lasagna/ 1
https://www.allrecipes.com/recipe/25303/chicken-fettuccini-alfredo/ 1
https://www.allrecipes.com/recipe/16723/pesto-chicken-florentine/ 1
https://www.allrecipes.com/recipe/11973/spaghetti-carbonara-ii/ 1
https://www.allrecipes.com/recipe/23431/to-die-for-fettuccine-alfredo/ 1
htt

https://www.allrecipes.com/recipe/87025/best-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/16433/stroganoff/ 1
https://www.allrecipes.com/recipe/16105/beef-stroganoff-ii/ 1
https://www.allrecipes.com/recipe/14626/ground-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/220059/chef-johns-classic-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/11880/easy-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/16311/simple-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/219046/rich-and-creamy-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/16196/darias-slow-cooker-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/23260/simple-hamburger-stroganoff/ 1
https://www.allrecipes.com/recipe/25202/beef-stroganoff-iii/ 1
https://www.allrecipes.com/recipe/16352/slow-cooker-beef-stroganoff-i/ 1
https://www.allrecipes.com/recipe/262062/our-favorite-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/262061/one-pot-ground-beef-stroganoff/ 1
https://www.allrecipes.com/recipe/24

https://www.allrecipes.com/recipe/86828/grilled-chicken-thighs-tandoori/ 1
https://www.allrecipes.com/recipe/14035/aloo-phujia/ 1
https://www.allrecipes.com/recipe/239867/chef-johns-chicken-tikka-masala/ 1
https://www.allrecipes.com/recipe/47564/baingan-bharta-eggplant-curry/ 1
https://www.allrecipes.com/recipe/174543/slow-cooker-butter-chicken/ 1
https://www.allrecipes.com/recipe/34689/chickpea-curry/ 1
https://www.allrecipes.com/recipe/165190/spicy-vegan-potato-curry/ 1
https://www.allrecipes.com/recipe/141169/easy-indian-butter-chicken/ 1
https://www.allrecipes.com/recipe/45957/chicken-makhani-indian-butter-chicken/ 1
https://www.allrecipes.com/recipe/16641/red-lentil-curry/ 1
https://www.allrecipes.com/recipe/46822/indian-chicken-curry-ii/ 1
https://www.allrecipes.com/recipe/60598/vegetarian-korma/ 1
https://www.allrecipes.com/recipe/228293/curry-stand-chicken-tikka-masala-sauce/ 1
https://www.allrecipes.com/recipe/45736/chicken-tikka-masala/ 1
https://www.allrecipes.com/recipe/254

https://www.allrecipes.com/recipe/222350/authentic-pad-thai/ 1
https://www.allrecipes.com/recipe/186965/slow-cooker-thai-peanut-pork/ 1
https://www.allrecipes.com/recipe/76778/slow-cook-thai-chicken/ 1
https://www.allrecipes.com/recipe/85815/thai-red-chicken-curry/ 1
https://www.allrecipes.com/recipe/141833/thai-green-curry-chicken/ 1
https://www.allrecipes.com/recipe/17511/chicken-satay/ 1
https://www.allrecipes.com/recipe/42968/pad-thai/ 1
https://www.allrecipes.com/recipe/257938/spicy-thai-basil-chicken-pad-krapow-gai/ 1
https://www.allrecipes.com/recipe/86005/thai-pineapple-chicken-curry/ 1
https://www.allrecipes.com/recipe/83083/thai-pork-with-peanut-sauce/ 1
https://www.allrecipes.com/recipe/72717/slow-cooker-thai-pork-with-rice/ 1
https://www.allrecipes.com/recipe/68813/spicy-basil-chicken/ 1
https://www.allrecipes.com/recipe/68532/curried-coconut-chicken/ 1
https://www.allrecipes.com/recipe/56094/thai-chicken-curry-in-coconut-milk/ 1
https://www.allrecipes.com/recipe/35892/thai

https://www.allrecipes.com/recipe/219646/authentic-mexican-chili-rellenos/ 1
https://www.allrecipes.com/recipe/259999/cochinita-pibil-mexican-pulled-pork-in-annatto-sauce/ 1
https://www.allrecipes.com/recipe/262261/mexican-corn-bread-casserole/ 1
https://www.allrecipes.com/recipe/258250/jeannies-vegetarian-enchiladas/ 1
https://www.allrecipes.com/recipe/239786/crispy-pork-carnitas/ 1
https://www.allrecipes.com/recipe/263122/best-fish-tacos/ 1
https://www.allrecipes.com/recipe/263162/low-carb-jicama-tostadas/ 1
https://www.allrecipes.com/recipe/222014/pantry-raid-chicken-enchilada-casserole/ 1
https://www.allrecipes.com/recipes/17135/world-cuisine/asian/chinese/main-dishes/ 0
https://www.allrecipes.com/recipe/49014/ten-minute-szechuan-chicken/ 1
https://www.allrecipes.com/recipe/13199/wonton-soup/ 1
https://www.allrecipes.com/recipe/146579/general-tao-chicken/ 1
https://www.allrecipes.com/recipe/195083/slow-cooker-mongolian-beef/ 1
https://www.allrecipes.com/recipe/17161/chinese-chicken

In [10]:
len(html_data)

1196

In [11]:
# Guardo los datos
with open("datasets/pickles/recipes_main_dish_main_categories.p", "wb") as p:
    pickle.dump(html_data, p)

### Categorias
Lista de categorias
Usando una libreria nueva se pueden conseguir más listas

In [3]:
# Category page (should make something to list categories)
html = gets_html("https://www.allrecipes.com/recipes/")
soup = BeautifulSoup(html)
# Find links
links = soup.find_all('a',href=True)
# Convert to string 
links = [str(i) for i in links]

link_regex = 'href=\"https://www.allrecipes.com/recipes/\d+/'
link_regex = re.compile(link_regex)

scrap_categories = []
# Filter by regex
links2 = list(filter(link_regex.search, links))
links2 = list(set(links2))

for li in links2:
    link_tmp = re.compile('href="(.+?)"').findall(li)[0]
    scrap_categories.append((link_tmp, 0))
    
    print("2",link_tmp)

2 https://www.allrecipes.com/recipes/22908/everyday-cooking/special-collections/new/?sort=newest
2 https://www.allrecipes.com/recipes/205/meat-and-poultry/pork/
2 https://www.allrecipes.com/recipes/362/desserts/cookies/
2 https://www.allrecipes.com/recipes/15172/fruits-and-vegetables/mushrooms/
2 https://www.allrecipes.com/recipes/78/breakfast-and-brunch/
2 https://www.allrecipes.com/recipes/22908/everyday-cooking/special-collections/new/?sort=newest
2 https://www.allrecipes.com/recipes/16791/everyday-cooking/special-collections/web-show-recipes/food-wishes/
2 https://www.allrecipes.com/recipes/95/pasta-and-noodles/
2 https://www.allrecipes.com/recipes/78/breakfast-and-brunch/
2 https://www.allrecipes.com/recipes/85/holidays-and-events/
2 https://www.allrecipes.com/recipes/206/meat-and-poultry/turkey/
2 https://www.allrecipes.com/recipes/1227/everyday-cooking/vegan/
2 https://www.allrecipes.com/recipes/16791/everyday-cooking/special-collections/web-show-recipes/food-wishes/
2 https://w